In [1]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal
import networkx as nx
import osmnx as ox
import time
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString, Polygon
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
from itertools import product, combinations
import multiprocessing as mp
import math
import warnings

In [2]:
# Block 1 state your assumptions
start = time.time()

# Assumptions
thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m
walkable_park_dist = 500 # radius in metres
one_park_buffer = 25 # in metres
park_entry_point_buffer = 25 # in metres
min_park_size = 400 # in squared metres (WHO = 0.04 ha = 400m)
park_entry_point_merge = 0

cities = ['Dublin, Ireland']

thresholds
# 'Dublin, Ireland' is problematic; road system is too dense.

[300, 600, 1000]

In [3]:
# Block 2 road networks

warnings.filterwarnings("ignore")

start_time = time.time()
graphs = list()
road_nodes = list()
road_edges = list()
road_conn = list()
roads = list()

for i in range(len(cities)):
    # Get graph, road nodes and edges
    graph = ox.graph_from_place(cities[i], network_type="all", buffer_dist = (np.max(thresholds)+1000))
    graphs.append(graph)
    road_node, road_edge = ox.graph_to_gdfs(graph)
    
    # Road nodes format
    road_node = road_node.to_crs(4326)
    road_node['geometry_m'] = gpd.GeoSeries(road_node['geometry'], crs = 4326).to_crs(3043)
    road_nodes.append(road_node)
    road = road_node.reset_index()
    roads.append(road)
    
    # format road edges
    road_edge = road_edge.to_crs(4326)
    road_edge['geometry_m'] = gpd.GeoSeries(road_edge['geometry'], crs = 4326).to_crs(3043)
    road_edge = road_edge.reset_index()
    road_edge.rename(columns={'u':'from', 'v':'to'}, inplace=True)
    road_edge['key'] = road_edge['from'].astype(str) + '-' + road_edge['to'].astype(str)
    road_edges.append(road_edge)
    
    # Get only necessary road connections columns for network performance
    road_con = road_edge[['osmid','key','length','geometry']]
    road_con = road_con.set_index('key')
    road_conn.append(road_con)
    print(cities[i].rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')
road_edges[0]

Dublin done 1.24 mns


,from,to,key,osmid,oneway,lanes,ref,name,highway,maxspeed,length,geometry,bridge,junction,width,access,tunnel,service,est_width,geometry_m
0,385708,446897184,385708-446897184,874491383,True,4,M50,Northern Cross Route Motorway,motorway,100,291.047,"LINESTRING (-6.35371 53.38880, -6.35216 53.389...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-121270.926 5956385.543, -121159.5..."
1,385807,385847,385807-385847,"[260858268, 548551653]",True,4,M50,Northern Cross Route,motorway,100,1592.269,"LINESTRING (-6.30155 53.40695, -6.29900 53.407...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-117550.766 5957940.623, -117375.8..."
2,385847,650846838,385847-650846838,"[98479817, 435558718]",True,"[2, 1]",NaN,NaN,motorway_link,100,571.998,"LINESTRING (-6.27774 53.40866, -6.27581 53.408...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-115949.546 5957923.332, -115819.8..."
3,385847,385848,385847-385848,"[260858267, 223780733]",True,3,M50,Northern Cross Route,motorway,100,1386.103,"LINESTRING (-6.27774 53.40866, -6.27492 53.408...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-115949.546 5957923.332, -115760.4..."
4,385848,385851,385848-385851,260858265,True,4,M50,Northern Cross Route,motorway,100,1218.036,"LINESTRING (-6.25689 53.40958, -6.25329 53.409...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-114555.254 5957844.273, -114314.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173951,9767043608,4446498670,9767043608-4446498670,341742993,False,NaN,NaN,NaN,service,NaN,61.734,"LINESTRING (-6.17766 53.37779, -6.17786 53.377...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-109765.872 5953637.928, -109776.4..."
173952,9767043608,4446498669,9767043608-4446498669,447555617,False,NaN,NaN,NaN,service,NaN,14.481,"LINESTRING (-6.17766 53.37779, -6.17750 53.37787)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-109765.872 5953637.928, -109753.4..."
173953,9767043609,4446503612,9767043609-4446503612,22664398,False,2,NaN,All Saints Park,residential,30,60.683,"LINESTRING (-6.17834 53.37746, -6.17776 53.37704)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-109815.309 5953607.151, -109782.8..."
173954,9767043609,4446498670,9767043609-4446498670,447555617,False,NaN,NaN,NaN,service,NaN,39.176,"LINESTRING (-6.17834 53.37746, -6.17788 53.37769)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-109815.309 5953607.151, -109781.8..."


In [4]:
# Block 3 city boundaries

# Get city boundaries
warnings.filterwarnings("ignore")
bound_df = pd.DataFrame()
for i in range(len(cities)):
    bound_df = pd.concat([bound_df, pd.DataFrame(ox.geocoder.geocode_to_gdf(cities[i]))])
    
bound_df = bound_df.reset_index()
bound_df = bound_df.loc[:,bound_df.columns!='index']
bound_df = gpd.GeoDataFrame(bound_df, geometry = 'geometry', crs = 4326)
bound_df.to_file(r'C:\Users\bartb\Downloads\bounds.shp')
bound_df

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((-6.38703 53.34081, -6.38658 53.33867...",53.410542,53.298734,-6.114883,-6.387026,282061541,relation,1109531,53.349764,-6.260273,"Dublin, Dublin 1, Leinster, Ireland",boundary,administrative,0.92159


In [5]:
# Block 4 city grids

# Get the grids
grids = list()
for i in range(len(cities)):
    # extract grids
    path = 'D:/Dumps/Population_grids/' + cities[i].rsplit(',')[0] + '/' + cities[i].rsplit(',')[0] + '_CPoPGrid.gpkg'
    file = gpd.read_file(path).to_crs(4326)

    # Get grid and boundary overlay
    popgrid = file.overlay(bound_df.iloc[i:i+1])

    # Only get full grids (area differs slightly due to lon-lat placement) and grids with people in it.
    popgrid['area'] = popgrid.area / popgrid.area.max()
    popgrid = popgrid[(popgrid['area'] >= 0.99)]
    popgrid = popgrid[popgrid['PoP2015_Number'] > 0]
    
    # Get grids centroids and lon-lat
    popgrid['centroid'] = popgrid.to_crs(4326).centroid
    popgrid['centroid_m'] = popgrid['centroid'].to_crs(3043)
    popgrid['grid_lon'] = popgrid['centroid_m'].x
    popgrid['grid_lat'] = popgrid['centroid_m'].y
    
    popgrid = popgrid.reset_index()
    grids.append(popgrid)
grids[0]

,index,grid_id,PoP2015_Number,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,...,display_name,class,type,importance,geometry,area,centroid,centroid_m,grid_lon,grid_lat
0,9,182,4,53.410542,53.298734,-6.114883,-6.387026,282061541,relation,1109531,...,"Dublin, Dublin 1, Leinster, Ireland",boundary,administrative,0.92159,"POLYGON ((-6.22296 53.40960, -6.22296 53.40826...",0.997469,POINT (-6.22409 53.40893),POINT (-112392.449 5957488.796),-112392.448934,5.957489e+06
1,10,183,4,53.410542,53.298734,-6.114883,-6.387026,282061541,relation,1109531,...,"Dublin, Dublin 1, Leinster, Ireland",boundary,administrative,0.92159,"POLYGON ((-6.22072 53.40960, -6.22072 53.40826...",0.997469,POINT (-6.22184 53.40893),POINT (-112243.732 5957469.407),-112243.732090,5.957469e+06
2,11,184,4,53.410542,53.298734,-6.114883,-6.387026,282061541,relation,1109531,...,"Dublin, Dublin 1, Leinster, Ireland",boundary,administrative,0.92159,"POLYGON ((-6.21847 53.40960, -6.21847 53.40826...",0.997469,POINT (-6.21960 53.40893),POINT (-112095.015 5957450.023),-112095.014961,5.957450e+06
3,12,185,63,53.410542,53.298734,-6.114883,-6.387026,282061541,relation,1109531,...,"Dublin, Dublin 1, Leinster, Ireland",boundary,administrative,0.92159,"POLYGON ((-6.21623 53.40960, -6.21623 53.40826...",0.997469,POINT (-6.21735 53.40893),POINT (-111946.298 5957430.643),-111946.297549,5.957431e+06
4,13,186,79,53.410542,53.298734,-6.114883,-6.387026,282061541,relation,1109531,...,"Dublin, Dublin 1, Leinster, Ireland",boundary,administrative,0.92159,"POLYGON ((-6.21398 53.40960, -6.21398 53.40826...",0.997469,POINT (-6.21510 53.40893),POINT (-111797.580 5957411.268),-111797.579852,5.957411e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4781,5465,7617,175,53.410542,53.298734,-6.114883,-6.387026,282061541,relation,1109531,...,"Dublin, Dublin 1, Leinster, Ireland",boundary,administrative,0.92159,"POLYGON ((-6.28809 53.30480, -6.28809 53.30346...",0.999937,POINT (-6.28921 53.30413),POINT (-118229.306 5946439.388),-118229.305863,5.946439e+06
4782,5474,7708,53,53.410542,53.298734,-6.114883,-6.387026,282061541,relation,1109531,...,"Dublin, Dublin 1, Leinster, Ireland",boundary,administrative,0.92159,"POLYGON ((-6.29258 53.30346, -6.29258 53.30211...",0.999968,POINT (-6.29371 53.30279),POINT (-118547.028 5946329.419),-118547.028120,5.946329e+06
4783,5475,7709,79,53.410542,53.298734,-6.114883,-6.387026,282061541,relation,1109531,...,"Dublin, Dublin 1, Leinster, Ireland",boundary,administrative,0.92159,"POLYGON ((-6.29034 53.30346, -6.29034 53.30211...",0.999968,POINT (-6.29146 53.30279),POINT (-118397.943 5946309.860),-118397.943224,5.946310e+06
4784,5476,7710,85,53.410542,53.298734,-6.114883,-6.387026,282061541,relation,1109531,...,"Dublin, Dublin 1, Leinster, Ireland",boundary,administrative,0.92159,"POLYGON ((-6.28809 53.30346, -6.28809 53.30211...",0.999968,POINT (-6.28921 53.30279),POINT (-118248.858 5946290.305),-118248.858045,5.946290e+06


In [6]:
# Block 5 city greenspace

parks_in_range = list()
for i in range(len(cities)):
    path = 'D:/Dumps/Greenspace/' + cities[i].rsplit(',')[0] + '/' + cities[i].rsplit(',')[0] + '_Greenspace.gpkg'
    greenspace = gpd.read_file(path).to_crs(4326)

    warnings.filterwarnings("ignore")

    green_buffer = gpd.GeoDataFrame(geometry = greenspace.to_crs(3043).buffer(one_park_buffer).to_crs(4326))
    greenspace['geometry_w_buffer'] = green_buffer
    greenspace['geometry_w_buffer'] = gpd.GeoSeries(greenspace['geometry_w_buffer'], crs = 4326)
    greenspace['geom buffer diff'] = greenspace['geometry_w_buffer'].difference(greenspace['geometry'])

    # This function group components in itself that overlap (with the buffer set of 25 metres)
    # https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
    W = libpysal.weights.fuzzy_contiguity(greenspace['geometry_w_buffer'])
    greenspace['components'] = W.component_labels
    parks = greenspace.dissolve('components')

    # Exclude parks below 0.04 ha.
    parks = parks[parks.to_crs(3043).area > min_park_size]
        
    # Get the park buffer
    parks_t = pd.DataFrame(parks.index).set_index('components')
    parks_t['park_geom'] = parks['geometry']
    parks_t = gpd.GeoDataFrame(parks_t, geometry = 'park_geom', crs = 4326)
    for j in range(len(thresholds)):
        Bound_buffer = gpd.GeoDataFrame(
            geometry = np.repeat(bound_df.iloc[i:i+1].to_crs(3043).
                                 buffer(thresholds[j]).to_crs(4326),len(parks)), 
            crs = 4326).reset_index().iloc[:,1]
        Bound_buffer.index = parks.index
        parks_t = parks_t.join(Bound_buffer, how = 'outer')
        parks_t = parks_t.rename(columns = {'geometry':('buffer_' + str(thresholds[j]))})
        
        # Check which parks intersect with the city boundary with buffer.
        parks_within_range = parks_t[parks_t['park_geom'].intersection(parks_t[('buffer_' + str(thresholds[j]))]).area > 0]
        pwr = pd.DataFrame(parks_within_range.index).set_index('components')
        pwr[str(thresholds[j])] = 1
        parks_t = parks_t.join(pwr, how = 'left')
        parks_t[str(thresholds[j])] = parks_t[str(thresholds[j])].fillna(0)
    parks_t = parks_t.join(parks)
    
    thresholds_str = list()
    for k in range(len(thresholds)): thresholds_str.append(str(thresholds[k]))
    parks_t = parks_t[(parks_t[thresholds_str] == 1).any(axis = 1)]
    parks_t = parks_t.reset_index()
    parks_t['geometry_m'] = parks_t['geometry'].to_crs(3043)
    parks_in_range.append(parks_t)
    
parks_in_range[0]

,components,park_geom,buffer_300,300,buffer_600,600,buffer_1000,1000,geometry,PRIMARYINDEX,...,Seven_a_Side,Rugby,Hard_Landscape,Committee_Area,Area_mSq,Area_Acre,Multipolygon,geometry_w_buffer,geom buffer diff,geometry_m
0,0,"POLYGON ((-6.20969 53.40416, -6.20970 53.40412...","POLYGON ((-6.39151 53.34086, -6.39149 53.34131...",1,"POLYGON ((-6.39599 53.34092, -6.39598 53.34137...",1,"POLYGON ((-6.40197 53.34100, -6.40195 53.34144...",1,"POLYGON ((-6.20969 53.40416, -6.20970 53.40412...",1223,...,None,None,None,None,1874.958,0.463,None,"POLYGON ((-6.21004 53.40423, -6.20991 53.40448...","POLYGON ((-6.20991 53.40448, -6.20990 53.40450...","POLYGON ((-111507.834 5956835.335, -111509.227..."
1,1,"MULTIPOLYGON (((-6.25231 53.34818, -6.25231 53...","POLYGON ((-6.39151 53.34086, -6.39149 53.34131...",1,"POLYGON ((-6.39599 53.34092, -6.39598 53.34137...",1,"POLYGON ((-6.40197 53.34100, -6.40195 53.34144...",1,"MULTIPOLYGON (((-6.25231 53.34818, -6.25231 53...",188,...,None,None,yes,Central Area,1992.119,0.492,YES,"POLYGON ((-6.24932 53.34847, -6.24932 53.34848...","POLYGON ((-6.24932 53.34848, -6.24933 53.34850...","MULTIPOLYGON (((-115141.559 5951000.061, -1151..."
2,2,"MULTIPOLYGON (((-6.31789 53.39298, -6.31790 53...","POLYGON ((-6.39151 53.34086, -6.39149 53.34131...",1,"POLYGON ((-6.39599 53.34092, -6.39598 53.34137...",1,"POLYGON ((-6.40197 53.34100, -6.40195 53.34144...",1,"MULTIPOLYGON (((-6.31789 53.39298, -6.31790 53...",1216,...,1,None,None,North West,1595.207,0.394,None,"POLYGON ((-6.31692 53.39486, -6.31655 53.39475...","POLYGON ((-6.31655 53.39475, -6.31654 53.39474...","MULTIPOLYGON (((-118836.654 5956535.394, -1188..."
3,3,"POLYGON ((-6.28609 53.37923, -6.28610 53.37923...","POLYGON ((-6.39151 53.34086, -6.39149 53.34131...",1,"POLYGON ((-6.39599 53.34092, -6.39598 53.34137...",1,"POLYGON ((-6.40197 53.34100, -6.40195 53.34144...",1,"POLYGON ((-6.28609 53.37923, -6.28610 53.37923...",1214,...,None,None,None,None,2799.290,0.692,None,"POLYGON ((-6.28636 53.37903, -6.28636 53.37904...","POLYGON ((-6.28636 53.37904, -6.28639 53.37906...","POLYGON ((-116930.184 5954734.930, -116931.079..."
4,4,"POLYGON ((-6.25867 53.39257, -6.25867 53.39257...","POLYGON ((-6.39151 53.34086, -6.39149 53.34131...",1,"POLYGON ((-6.39599 53.34092, -6.39598 53.34137...",1,"POLYGON ((-6.40197 53.34100, -6.40195 53.34144...",1,"POLYGON ((-6.25867 53.39257, -6.25867 53.39257...",1209,...,None,None,None,None,1032.706,0.255,None,"POLYGON ((-6.25942 53.39220, -6.25943 53.39222...","POLYGON ((-6.25943 53.39222, -6.25943 53.39224...","POLYGON ((-114920.008 5955974.445, -114920.068..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,352,"POLYGON ((-6.22964 53.32212, -6.22954 53.32210...","POLYGON ((-6.39151 53.34086, -6.39149 53.34131...",1,"POLYGON ((-6.39599 53.34092, -6.39598 53.34137...",1,"POLYGON ((-6.40197 53.34100, -6.40195 53.34144...",1,"POLYGON ((-6.22964 53.32212, -6.22954 53.32210...",209,...,None,None,None,None,39641.145,9.796,None,"POLYGON ((-6.23002 53.32213, -6.23001 53.32218...","POLYGON ((-6.23001 53.32218, -6.23001 53.32220...","POLYGON ((-114014.938 5947915.997, -114008.475..."
353,353,"POLYGON ((-6.23440 53.34106, -6.23435 53.34105...","POLYGON ((-6.39151 53.34086, -6.39149 53.34131...",1,"POLYGON ((-6.39599 53.34092, -6.39598 53.34137...",1,"POLYGON ((-6.40197 53.34100, -6.40195 53.34144...",1,"POLYGON ((-6.23440 53.34106, -6.23435 53.34105...",201,...,None,None,None,South East,3882.440,0.959,None,"POLYGON ((-6.23476 53.34111, -6.23465 53.34143...","POLYGON ((-6.23465 53.34143, -6.23465 53.34143...","POLYGON ((-114056.621 5950056.094, -114053.473..."
354,354,"POLYGON ((-6.23312 53.36771, -6.23310 53.36770...","POLYGON ((-6.39151 53.34086, -6.39149 53.34131...",1,"POLYGON ((-6.39599 53.34092, -6.39598 53.34137...",1,"POLYGON ((-6.40197 53.34100, -6.40195 53.34144...",1,"POLYGON ((-6.23312 53.36771, -6.23310 53.36770...",1227,...,None,None,None,None,3805.057,0.940,None,"POLYGON ((-6.23328 5

In [7]:
parks_in_range[0]['geometry_m'].to_file('D:/Dumps/Dublin_buf_parks.shp')

In [8]:
# Block 6 park entry points

start_time = time.time()
ParkRoads = list()
for j in range(len(cities)):
    ParkRoad = pd.DataFrame()
    mat = list()
    # For all
    for i in range(len(parks_in_range[j])):
        dist = road_nodes[j]['geometry'].to_crs(3043).distance(parks_in_range[j]['geometry'].to_crs(
            3043)[i])
        buf_nodes = road_nodes[j][(dist < park_entry_point_buffer) & (dist > 0)]
        mat.append(list(np.repeat(i, len(buf_nodes))))
        ParkRoad = pd.concat([ParkRoad, buf_nodes])
        if i % 50 == 0: print(cities[j].rsplit(',')[0], round(i/len(parks_in_range[j])*100,1),'% done', 
                              round((time.time() - start_time) / 60,2),' mns')
    # Park no list conversion
    mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

    # Format
    ParkRoad['Park_No'] = mat_u
    ParkRoad = ParkRoad.reset_index()
    ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
    ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
    
    # Get the road nodes intersecting with the parks' buffer
    ParkRoad = pd.merge(ParkRoad, parks_in_range[j][['park_geom','geometry']], left_on = 'Park_No', right_index = True)
    ParkRoad = pd.merge(ParkRoad, parks_in_range[j][thresholds_str], left_on = 'Park_No', right_index = True)
    
    # Get the walkable park size
    ParkRoad['park_size_walkable'] = ParkRoad['geometry_m'].buffer(walkable_park_dist).to_crs(4326).intersection(ParkRoad['geometry_y'])
    ParkRoad['walk_area'] = ParkRoad['park_size_walkable'].to_crs(3043).area
    ParkRoad['park_area'] = ParkRoad['geometry_y'].to_crs(3043).area
    ParkRoad['share_walked'] = ParkRoad['walk_area'] / ParkRoad['park_area']
    
    # Get size inflation factors for the gravity model
    ParkRoad['size_infl_factor'] = ParkRoad['walk_area'] / ParkRoad['walk_area'].median()
    ParkRoad['size_infl_proot2'] = ParkRoad['size_infl_factor']**(1/2)
    ParkRoad['size_infl_proot3'] = ParkRoad['size_infl_factor']**(1/3)
    ParkRoad['size_infl_proot5'] = ParkRoad['size_infl_factor']**(1/5)    
    ParkRoads.append(ParkRoad)
    
    print(cities[j].rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')

ParkRoads[0]

Dublin 0.0 % done 0.0  mns
Dublin 14.0 % done 0.35  mns
Dublin 28.0 % done 0.79  mns
Dublin 42.0 % done 1.14  mns
Dublin 56.0 % done 1.43  mns
Dublin 70.0 % done 1.72  mns
Dublin 84.0 % done 2.07  mns
Dublin 98.0 % done 2.36  mns
Dublin done 2.44 mns


,osmid,y,x,street_count,ref,highway,geometry_x,geometry_m,Park_No,park_lon,...,600,1000,park_size_walkable,walk_area,park_area,share_walked,size_infl_factor,size_infl_proot2,size_infl_proot3,size_infl_proot5
0,305252219,53.404449,-6.209373,1,NaN,turning_circle,POINT (-6.20937 53.40445),POINT (-111482.718 5956865.176),0,-111482.717972,...,1,1,"POLYGON ((-6.20969 53.40416, -6.20955 53.40441...",1890.710860,1890.710860,1.000000,0.072480,0.269220,0.416939,0.591621
1,389292,53.348280,-6.254945,4,J41,traffic_signals,POINT (-6.25494 53.34828),POINT (-115314.902 5951034.196),1,-115314.902187,...,1,1,"MULTIPOLYGON (((-6.24752 53.34779, -6.24776 53...",12220.154412,37710.095438,0.324055,0.468455,0.684438,0.776645,0.859278
2,26770318,53.347622,-6.241187,4,J80,traffic_signals,POINT (-6.24119 53.34762),POINT (-114412.098 5950842.071),1,-114412.097779,...,1,1,"MULTIPOLYGON (((-6.24865 53.34792, -6.24863 53...",26364.138717,37710.095438,0.699127,1.010659,1.005315,1.003540,1.002123
3,32476190,53.348051,-6.247891,4,NaN,crossing,POINT (-6.24789 53.34805),POINT (-114850.447 5950947.744),1,-114850.446642,...,1,1,"MULTIPOLYGON (((-6.24108 53.34989, -6.24100 53...",34152.769593,37710.095438,0.905666,1.309233,1.144217,1.093971,1.055367
4,111887238,53.350406,-6.245636,3,NaN,NaN,POINT (-6.24564 53.35041),POINT (-114666.828 5951189.191),1,-114666.828257,...,1,1,"MULTIPOLYGON (((-6.24013 53.34738, -6.24046 53...",33413.142758,37710.095438,0.886053,1.280880,1.131760,1.086016,1.050756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4804,6646353552,53.398074,-6.236181,1,NaN,NaN,POINT (-6.23618 53.39807),POINT (-113350.389 5956390.058),356,-113350.388587,...,1,1,"POLYGON ((-6.23304 53.40213, -6.23311 53.40207...",17882.107493,18378.786640,0.972975,0.685504,0.827952,0.881732,0.927261
4805,6646353553,53.397812,-6.236181,3,NaN,NaN,POINT (-6.23618 53.39781),POINT (-113354.183 5956360.989),356,-113354.183465,...,1,1,"POLYGON ((-6.23311 53.40189, -6.23309 53.40187...",17466.422971,18378.786640,0.950358,0.669569,0.818272,0.874846,0.922909
4806,6646353558,53.398474,-6.236213,1,NaN,NaN,POINT (-6.23621 53.39847),POINT (-113346.737 5956434.643),356,-113346.737179,...,1,1,"POLYGON ((-6.23278 53.40245, -6.23279 53.40244...",18344.763028,18378.786640,0.998149,0.703239,0.838594,0.889272,0.932010
4807,6646353559,53.398373,-6.236192,1,NaN,NaN,POINT (-6.23619 53.39837),POINT (-113346.778 5956423.287),356,-113346.778081,...,1,1,"POLYGON ((-6.23287 53.40238, -6.23281 53.40236...",18258.004729,18378.786640,0.993428,0.699914,0.836608,0.887867,0.931127


In [9]:
# Get buffer of nodes close to each other.
ParkCombs = list([])
for i in range(len(cities)):
    ParkComb = ParkRoads[i]
    ParkComb['geometry_m_buffer'] = ParkComb['geometry_m'].buffer(park_entry_point_merge)
    print(1)
    M = libpysal.weights.fuzzy_contiguity(ParkComb['geometry_m_buffer'])
    print(2)
    ParkComb['components'] = M.component_labels
    centr = gpd.GeoDataFrame(ParkComb, geometry = 'geometry_x', crs = 4326).dissolve('components')['geometry_x'].centroid
    centr = gpd.GeoDataFrame(centr)
    centr.columns = ['comp_centroid']
    ParkComb = pd.merge(ParkComb, centr, left_on = 'components', right_index = True)
    ParkComb['centr_dist'] = ParkComb['geometry_x'].distance(ParkComb['comp_centroid'])
    ParkComb = ParkComb.iloc[ParkComb.groupby('components')['centr_dist'].idxmin()]
    ParkCombs.append(ParkComb)
ParkCombs[0]

1
2


,osmid,y,x,street_count,ref,highway,geometry_x,geometry_m,Park_No,park_lon,...,park_area,share_walked,size_infl_factor,size_infl_proot2,size_infl_proot3,size_infl_proot5,geometry_m_buffer,components,comp_centroid,centr_dist
0,305252219,53.404449,-6.209373,1,NaN,turning_circle,POINT (-6.20937 53.40445),POINT (-111482.718 5956865.176),0,-111482.717972,...,1890.710860,1.000000,0.072480,0.269220,0.416939,0.591621,POLYGON EMPTY,0,POINT (-6.20937 53.40445),0.0
1,389292,53.348280,-6.254945,4,J41,traffic_signals,POINT (-6.25494 53.34828),POINT (-115314.902 5951034.196),1,-115314.902187,...,37710.095438,0.324055,0.468455,0.684438,0.776645,0.859278,POLYGON EMPTY,1,POINT (-6.25494 53.34828),0.0
2,26770318,53.347622,-6.241187,4,J80,traffic_signals,POINT (-6.24119 53.34762),POINT (-114412.098 5950842.071),1,-114412.097779,...,37710.095438,0.699127,1.010659,1.005315,1.003540,1.002123,POLYGON EMPTY,2,POINT (-6.24119 53.34762),0.0
3,32476190,53.348051,-6.247891,4,NaN,crossing,POINT (-6.24789 53.34805),POINT (-114850.447 5950947.744),1,-114850.446642,...,37710.095438,0.905666,1.309233,1.144217,1.093971,1.055367,POLYGON EMPTY,3,POINT (-6.24789 53.34805),0.0
4,111887238,53.350406,-6.245636,3,NaN,NaN,POINT (-6.24564 53.35041),POINT (-114666.828 5951189.191),1,-114666.828257,...,37710.095438,0.886053,1.280880,1.131760,1.086016,1.050756,POLYGON EMPTY,4,POINT (-6.24564 53.35041),0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4804,6646353552,53.398074,-6.236181,1,NaN,NaN,POINT (-6.23618 53.39807),POINT (-113350.389 5956390.058),356,-113350.388587,...,18378.786640,0.972975,0.685504,0.827952,0.881732,0.927261,POLYGON EMPTY,4804,POINT (-6.23618 53.39807),0.0
4805,6646353553,53.397812,-6.236181,3,NaN,NaN,POINT (-6.23618 53.39781),POINT (-113354.183 5956360.989),356,-113354.183465,...,18378.786640,0.950358,0.669569,0.818272,0.874846,0.922909,POLYGON EMPTY,4805,POINT (-6.23618 53.39781),0.0
4806,6646353558,53.398474,-6.236213,1,NaN,NaN,POINT (-6.23621 53.39847),POINT (-113346.737 5956434.643),356,-113346.737179,...,18378.786640,0.998149,0.703239,0.838594,0.889272,0.932010,POLYGON EMPTY,4806,POINT (-6.23621 53.39847),0.0
4807,6646353559,53.398373,-6.236192,1,NaN,NaN,POINT (-6.23619 53.39837),POINT (-113346.778 5956423.287),356,-113346.778081,...,18378.786640,0.993428,0.699914,0.836608,0.887867,0.931127,POLYGON EMPTY,4807,POINT (-6.23619 53.39837),0.0


In [10]:
# Block 7 grid-parkentry combinations within euclidean threshold distance

start_time = time.time()
RoadComb = list()
for l in range(len(cities)):
    print(cities[l])
    # Check all parks within 1000m radius
    len1 = len(grids[l])
    len2 = len(ParkCombs[l])
    len3 = int(np.ceil(len2/1000))
    output = pd.DataFrame()
    len_mat = 0
    # Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
    for i in range(len3):
        # Check all grid-park combinations per 1000
        l1, l2 = range(0,len1), range(i*1000,(i+1)*1000)
        listed = pd.DataFrame(list(product(l1, l2)))

        # Merge grid and park information
        grid_merged = pd.merge(listed, 
                               grids[l][['grid_id','grid_lon','grid_lat','centroid','centroid_m']],
                               left_on = 0, right_index = True)
        node_merged = pd.merge(grid_merged, 
                               ParkCombs[l][['Park_No','osmid','geometry_x','geometry_y','geometry_m','park_lon','park_lat',
                                   'size_infl_proot2','size_infl_proot3','size_infl_proot5','share_walked','park_area','walk_area']], 
                               left_on = 1, right_index = True)

        # Preset index for merging
        node_merged['key'] = range(0,len(node_merged))
        node_merged = node_merged.set_index('key')
        node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]

        # Create lists for better computational performance
        glon = list(node_merged['grid_lon'])
        glat = list(node_merged['grid_lat'])
        plon = list(node_merged['park_lon'])
        plat = list(node_merged['park_lat'])
        infl2 = list(node_merged['size_infl_proot2'])
        infl3 = list(node_merged['size_infl_proot3'])
        infl5 = list(node_merged['size_infl_proot5'])

        # Get the euclidean distances
        mat = list()
        mat2 = list()
        mat3 = list()
        mat4 = list()
        for j in range(len(node_merged)):
            mat.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2))
            mat2.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl2[j])
            mat3.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl3[j])
            mat4.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl5[j])

        # Check if distances are within 1000m and join remaining info and concat in master df per 1000.
        mat_df = pd.DataFrame(mat3)[(np.array(mat) <= np.max(thresholds)) | 
                                    (np.array(mat2) <= np.max(thresholds)) | 
                                    (np.array(mat3) <= np.max(thresholds)) | 
                                    (np.array(mat4) <= np.max(thresholds))]

        # join the other gravity euclidean scores and other information
        mat_df = mat_df.join(pd.DataFrame(mat), lsuffix='_infl', rsuffix='_entr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat2), lsuffix='_entry', rsuffix='_pwr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat4), lsuffix='_pwr', rsuffix='_root', how = 'left')
        mat_df.columns = ['size_infl_eucl2','raw euclidean','size_infl_eucl3','size_infl_eucl5']    
        mat_df = mat_df.join(node_merged)

        output = pd.concat([output, mat_df])

        print((i+1),'/',len3,'comb. done',round((time.time() - start_time) / 60,2),' mns')
        print('of',np.where(i+1 == len3, len2 % 1000 * 1000, len1*1000) ,'within a Gravity model variant in one of',
              thresholds,'m threshold:',len(mat_df))

        # Checks the number of the parks within 1000m.
        len_mat = len_mat + len(mat_df)

    # Renaming columns
    print('total combinations within distance',len_mat)
    
    output.columns = ['size_infl_eucl3','raw euclidean','size_infl_eucl2','size_infl_eucl5',
                      'Grid_No','Park_entry_No','Grid_Id','grid_lon','grid_lat','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Parkroad_coords_centroid','Park_geom','Parkroad_m_centroid',
                      'park_lon','park_lat','size_infl_proot2','size_infl_proot3','size_infl_proot5',
                      'parkshare_walked','park_area','walk_area_m2']
    output = output[['raw euclidean','size_infl_eucl2','size_infl_eucl3','size_infl_eucl5',
                     'Grid_No','Park_entry_No','Grid_Id','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Parkroad_coords_centroid','Park_geom','Parkroad_m_centroid',
                     'walk_area_m2','size_infl_proot2','size_infl_proot3','size_infl_proot5']]
    
    # Reinstate geographic elements
    output = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
    output['Grid_m_centroid'] = gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043)
    output['Parkroad_coords_centroid'] = gpd.GeoSeries(output['Parkroad_coords_centroid'], crs = 4326)
    output['Parkroad_m_centroid'] = gpd.GeoSeries(output['Parkroad_m_centroid'], crs = 3043)
    
    # Get the nearest entrance point for the grid centroids
    mat5 = list()
    for i in range(len(output)):
        nearest = int(roads[l]['geometry'].sindex.nearest(output['Grid_coords_centroid'].iloc[i])[1])
        mat5.append(roads[l]['osmid'].iloc[nearest])
        if i % 250000 == 0: print(round(i/len(output)*100,1),'% gridentry done', round((time.time() - start_time) / 60,2),' mns')
            
    # format resulting dataframe
    output['grid_osm'] = mat5
    output = pd.merge(output, road_nodes[l]['geometry'], left_on = 'grid_osm', right_index = True)
    output['geometry_m'] = gpd.GeoSeries(output['geometry'], crs = 4326).to_crs(3043)
    output['grid_entry_dist'] = round(gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043
                                                   ).distance(output['geometry_m']),3)
    output = output.reset_index()
    print('100 % gridentry done', round((time.time() - start_time) / 60,2),' mns')
    RoadComb.append(output)
RoadComb[0]


Dublin, Ireland
1 / 5 comb. done 0.54  mns
of 4786000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 337083
2 / 5 comb. done 1.09  mns
of 4786000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 834193
3 / 5 comb. done 1.66  mns
of 4786000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 276478
4 / 5 comb. done 2.19  mns
of 4786000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 199377
5 / 5 comb. done 2.65  mns
of 809000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 133627
total combinations within distance 1780758
0.0 % gridentry done 2.66  mns
14.0 % gridentry done 3.08  mns
28.1 % gridentry done 3.51  mns
42.1 % gridentry done 3.92  mns
56.2 % gridentry done 4.33  mns
70.2 % gridentry done 4.75  mns
84.2 % gridentry done 5.16  mns
98.3 % gridentry done 5.57  mns
100 % gridentry done 5.69  mns


,index,raw euclidean,size_infl_eucl2,size_infl_eucl3,size_infl_eucl5,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,...,Park_geom,Parkroad_m_centroid,walk_area_m2,size_infl_proot2,size_infl_proot3,size_infl_proot5,grid_osm,geometry,geometry_m,grid_entry_dist
0,1,971.718595,3609.378217,2330.603709,1642.469317,1,0,183,POINT (-6.22184 53.40893),POINT (-112243.732 5957469.407),...,"POLYGON ((-6.20969 53.40416, -6.20970 53.40412...",POINT (-111482.718 5956865.176),1890.710860,0.269220,0.416939,0.591621,822791806,POINT (-6.22185 53.40795),POINT (-112258.366 5957360.336),110.048
1,4360047,671.120029,226.785155,325.594066,434.836478,1,911,183,POINT (-6.22184 53.40893),POINT (-112243.732 5957469.407),...,"MULTIPOLYGON (((-6.18812 53.39655, -6.18809 53...",POINT (-111708.380 5957064.684),228444.145614,2.959277,2.061217,1.543385,822791806,POINT (-6.22185 53.40795),POINT (-112258.366 5957360.336),110.048
2,4364833,607.841672,215.266951,304.262477,401.296365,1,912,183,POINT (-6.22184 53.40893),POINT (-112243.732 5957469.407),...,"MULTIPOLYGON (((-6.18812 53.39655, -6.18809 53...",POINT (-111773.114 5957084.712),207986.477877,2.823665,1.997754,1.514695,822791806,POINT (-6.22185 53.40795),POINT (-112258.366 5957360.336),110.048
3,4369619,1279.857467,517.573506,699.898764,891.016566,1,913,183,POINT (-6.22184 53.40893),POINT (-112243.732 5957469.407),...,"MULTIPOLYGON (((-6.18812 53.39655, -6.18809 53...",POINT (-111096.884 5956901.274),159510.037174,2.472803,1.828632,1.436401,822791806,POINT (-6.22185 53.40795),POINT (-112258.366 5957360.336),110.048
4,4374405,732.878515,241.479443,349.620943,470.079444,1,914,183,POINT (-6.22184 53.40893),POINT (-112243.732 5957469.407),...,"MULTIPOLYGON (((-6.18812 53.39655, -6.18809 53...",POINT (-111640.764 5957052.825),240277.180442,3.034952,2.096209,1.559052,822791806,POINT (-6.22185 53.40795),POINT (-112258.366 5957360.336),110.048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1780753,2134555,2190.483565,910.609157,1220.116617,1541.902117,4785,4445,7802,POINT (-6.29146 53.30144),POINT (-118417.500 5946160.773),...,"MULTIPOLYGON (((-6.29390 53.32489, -6.29352 53...",POINT (-118179.856 5948338.327),150947.126289,2.405515,1.795307,1.420637,1425375093,POINT (-6.29133 53.30125),POINT (-118411.720 5946138.909),22.615
1780754,2168057,2347.591986,925.205033,1261.923023,1617.591163,4785,4452,7802,POINT (-6.29146 53.30144),POINT (-118417.500 5946160.773),...,"MULTIPOLYGON (((-6.29390 53.32489, -6.29352 53...",POINT (-118311.733 5948505.981),167949.278037,2.537375,1.860329,1.451289,1425375093,POINT (-6.29133 53.30125),POINT (-118411.720 5946138.909),22.615
1780755,2172843,2191.431686,916.231358,1225.310287,1546.104423,4785,4453,7802,POINT (-6.29146 53.30144),POINT (-118417.500 5946160.773),...,"MULTIPOLYGON (((-6.29390 53.32489, -6.29352 53...",POINT (-118219.060 5948343.201),149229.419167,2.391789,1.788471,1.417389,1425375093,POINT (-6.29133 53.30125),POINT (-118411.720 5946138.909),22.615
1780756,2182415,2537.503715,993.020502,1357.607950,1743.521464,4785,4455,7802,POINT (-6.29146 53.30144),POINT (-118417.500 5946160.773),...,"MULTIPOLYGON (((-6.29390 53.32489, -6.29352 53...",POINT (-118368.115 5948697.796),170335.755427,2.555339,1.869099,1.455390,1425375093,POINT (-6.29133 53.30125),POINT (-118411.720 5946138.909),22.615


In [11]:
for i in range(len(RoadComb)): print(cities[i], len(RoadComb[i]))

Dublin, Ireland 1780758


In [12]:
# Block 7 calculate route networks of all grid-parkentry combinations within euclidean threshold distance

warnings.filterwarnings("ignore")
start_time = time.time()

Routes = list()
for j in range(len(cities)):

    Graph = graphs[j]
    CityRoads = RoadComb[j] # iloc to test the iteration speed.
    PR = ParkRoads[j]
    
    block = 250000

    Route_parts = pd.DataFrame()
    len2 = int(np.ceil(len(CityRoads)/block))
    
    # Divide in chunks of 250000 for computational load
    for k in range(len2):    
        CityRoad = CityRoads.iloc[k*block:k*block+block]

        parknode = list(CityRoad['Parkroad_osmid'])
        gridnode = list(CityRoad['grid_osm'])
        
        s_mat = list([])
        s_mat1 = list([])
        s_mat2 = list([])
        s_mat3 = list([])
        s_mat4 = list([])
        
        len1 = len(CityRoad)
        
        print('range',k+1,'/',len2,':',k*block,'-',k*block+np.where(k*block+block >= len1,len1,block))
        for i in range(len(CityRoad)):
            try:
                shortest = nx.shortest_path(Graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
                s_mat.append(shortest)
                shortest_to = list(shortest[1:len(shortest)])
                shortest_to.append(0)
                s_mat1.append(shortest_to)
                s_mat2.append(list(np.repeat(i, len(shortest))))
                s_mat3.append(list(np.arange(0, len(shortest))))
                s_mat4.append('normal way')
            except:
                try:
                    # Check the reverse
                    shortest = nx.shortest_path(Graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
                    s_mat.append(shortest)
                    shortest_to = list(shortest[1:len(shortest)])
                    shortest_to.append(0)
                    s_mat1.append(shortest_to)
                    s_mat2.append(list(np.repeat(i, len(shortest))))
                    s_mat3.append(list(np.arange(0, len(shortest))))
                    s_mat4.append('reverse way')
                except:
                    # Otherwise the nearest node is taken, which is iterated 10 times at max. Order in route for nearest node:
                    # 1. gridnode to nearest to the original failed parknode
                    # 2. The reverse of 1.
                    # 3. nearest gridnode to the failed one and route to park
                    # 4. The reverse of 3.

                    len3 = 0
                    alt_route = list([])
                    while len3 < 10 and len(alt_route) < 1:

                        len3 = len3 +1
                        # Grid nearest
                        g_geom = gpd.GeoSeries(PR[PR['osmid'] == int(CityRoad.iloc[i:i+1]['grid_osm'])]['geometry_x'])
                        g_nearest = pd.DataFrame((abs(float(g_geom.x) - gpd.GeoSeries(PR['geometry_x']).x)**2
                        +abs(float(g_geom.y) - gpd.GeoSeries(PR['geometry_x']).y)**2)**(1/2)
                                                ).join(PR['osmid']).sort_values(0)

                        g_grid = g_nearest.iloc[len3,1]
                        g_park = CityRoad.iloc[i]['Parkroad_osmid']

                        p_geom = gpd.GeoSeries(PR[PR['osmid'] == int(CityRoad.iloc[i:i+1]['Parkroad_osmid'])]['geometry_x'])
                        p_nearest = pd.DataFrame((abs(float(p_geom.x) - gpd.GeoSeries(PR['geometry_x']).x)**2
                        +abs(float(p_geom.y) - gpd.GeoSeries(PR['geometry_x']).y)**2)**(1/2)
                                                ).join(PR['osmid']).sort_values(0)

                        p_grid = CityRoad.iloc[i]['grid_osm']
                        p_park = p_nearest.iloc[len3,1]

                        try:
                            alt_route.append(nx.shortest_path(graphs[0], p_grid, p_park, 'travel_dist', method = 'dijkstra'))
                        except:
                            try:
                                alt_route.append(nx.shortest_path(graphs[0], p_park, p_grid, 'travel_dist', method = 'dijkstra'))
                            except:
                                try:
                                    alt_route.append(nx.shortest_path(graphs[0], g_grid, g_park, 'travel_dist', method = 'dijkstra'))
                                except:
                                    try:
                                        alt_route.append(nx.shortest_path(graphs[0], g_grid, g_park, 'travel_dist', method = 'dijkstra'))
                                    except:
                                        if len3 == 10:
                                            print('No route between grid and park-entry point and both their 10 alternatives')
                                        pass
                    s_mat.append(alt_route[0])
                    shortest_to = list(alt_route[0][1:len(alt_route[0])])
                    shortest_to.append(0)
                    s_mat1.append(shortest_to)
                    s_mat2.append(list(np.repeat(i, len(alt_route[0]))))
                    s_mat3.append(list(np.arange(0, len(alt_route[0]))))
                    s_mat4.append('normal way')


            if i % 10000 == 0: print(cities[j].rsplit(',')[0], round(i/len(CityRoads)*100,2),'% done', 
                                        round((time.time() - start_time) / 60,2),'mns')

        print(cities[j].rsplit(',')[0], '100 % pathfinding done', round((time.time() - start_time) / 60,2),' mns')

        # Unpack lists
        s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat) for i in b]
        s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat1) for i in b]
        s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat2) for i in b]
        s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat3) for i in b]

        # Format df
        routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()
        routes.columns = ['from','to','route','step']
        mat_key = list([])
        for n in range(len(routes)):
            mat_key.append(str(int(s_mat_u[n])) + '-' + str(int(s_mat_u1[n])))
        routes['key'] = mat_key
        routes = routes.set_index('key')

        # Add route information
        routes = routes.join(road_conn[j], how = 'left')
        routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
        print(cities[j].rsplit(',')[0], 'formatting done', round((time.time() - start_time) / 60,2), 'mns')
        routes = routes.sort_values(by = ['route','step'])

        # get single (dissolved) line per route, attach information.
        routes2 = routes[['route','geometry']].dissolve('route')
        routes2['way_calculated'] = s_mat4
        routes2['route_cost'] = routes.groupby('route')['length'].sum()
        routes2['num_steps'] = routes.groupby('route')['step'].max()
        routes2['index'] = CityRoad.index
        routes2 = routes2.set_index(['index'])
        routes2.index = routes2.index.astype(int)
        routes2 = pd.merge(routes2, CityRoad[['Grid_No','Park_No','Park_entry_No','grid_entry_dist','Parkroad_osmid','grid_osm',
                                                'walk_area_m2','size_infl_proot2','size_infl_proot3', 'size_infl_proot5',
                                                'raw euclidean']],
                            left_index = True, right_index = True)
        routes2['raw_total_cost'] = routes2['route_cost'] + routes2['grid_entry_dist']
        routes2['grav2_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot2']
        routes2['grav3_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot3']
        routes2['grav5_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot5']

        routes2['gridpark_no'] = routes2['Grid_No'].astype(str) +'-'+ routes2['Park_No'].astype(str)
        print(cities[j].rsplit(',')[0], 'dissolving done', round((time.time() - start_time) / 60,2), 'mns')
        Route_parts = pd.concat([Route_parts, routes2])
    Routes.append(Route_parts)
Routes[0]

range 1 / 8 : 0 - 250000
Dublin 0.0 % done 0.0 mns
Dublin 0.56 % done 0.54 mns
Dublin 1.12 % done 1.21 mns
Dublin 1.68 % done 1.84 mns
Dublin 2.25 % done 4.57 mns
Dublin 2.81 % done 7.81 mns
Dublin 3.37 % done 10.9 mns
Dublin 3.93 % done 13.73 mns
Dublin 4.49 % done 16.45 mns
Dublin 5.05 % done 19.33 mns
Dublin 5.62 % done 22.41 mns
Dublin 6.18 % done 25.87 mns
Dublin 6.74 % done 28.58 mns
Dublin 7.3 % done 30.25 mns
Dublin 7.86 % done 32.76 mns
Dublin 8.42 % done 36.44 mns
Dublin 8.98 % done 38.76 mns
Dublin 9.55 % done 40.9 mns
Dublin 10.11 % done 43.0 mns
Dublin 10.67 % done 45.28 mns
Dublin 11.23 % done 48.02 mns
Dublin 11.79 % done 50.97 mns
Dublin 12.35 % done 54.14 mns
Dublin 12.92 % done 57.29 mns
Dublin 13.48 % done 60.45 mns
Dublin 100 % pathfinding done 63.63  mns
Dublin formatting done 67.81 mns
Dublin dissolving done 69.65 mns
range 2 / 8 : 250000 - 500000
Dublin 0.0 % done 69.68 mns
Dublin 0.56 % done 73.59 mns
Dublin 1.12 % done 78.57 mns
Dublin 1.68 % done 83.72 mns
Dub

,geometry,way_calculated,route_cost,num_steps,Grid_No,Park_No,Park_entry_No,grid_entry_dist,Parkroad_osmid,grid_osm,walk_area_m2,size_infl_proot2,size_infl_proot3,size_infl_proot5,raw euclidean,raw_total_cost,grav2_total_cost,grav3_total_cost,grav5_total_cost,gridpark_no
index,,,,,,,,,,,,,,,,,,,,
0,"MULTILINESTRING ((-6.22185 53.40795, -6.22201 ...",normal way,3435.650,43,1,0,0,110.048,305252219,822791806,1890.710860,0.269220,0.416939,0.591621,971.718595,3545.698,13170.237959,8504.125526,5993.196182,1-0
1,"MULTILINESTRING ((-6.22185 53.40795, -6.22201 ...",normal way,3073.529,39,1,47,911,110.048,291661207,822791806,228444.145614,2.959277,2.061217,1.543385,671.120029,3183.577,1075.795642,1544.513256,2062.724027,1-47
2,"MULTILINESTRING ((-6.22185 53.40795, -6.22201 ...",normal way,3141.032,40,1,47,912,110.048,291661210,822791806,207986.477877,2.823665,1.997754,1.514695,607.841672,3251.080,1151.369034,1627.367289,2146.359237,1-47
3,"MULTILINESTRING ((-6.22185 53.40795, -6.22201 ...",normal way,3743.302,46,1,47,913,110.048,291661486,822791806,159510.037174,2.472803,1.828632,1.436401,1279.857467,3853.350,1558.292170,2107.230668,2682.641446,1-47
4,"MULTILINESTRING ((-6.22185 53.40795, -6.22201 ...",normal way,3141.668,40,1,47,914,110.048,291661530,822791806,240277.180442,3.034952,2.096209,1.559052,732.878515,3251.716,1071.422551,1551.236654,2085.700177,1-47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1780753,"MULTILINESTRING ((-6.29133 53.30125, -6.29165 ...",normal way,3756.001,42,4785,318,4445,22.615,2992031602,1425375093,150947.126289,2.405515,1.795307,1.420637,2190.483565,3778.616,1570.814036,2104.718905,2659.803571,4785-318
1780754,"MULTILINESTRING ((-6.29133 53.30125, -6.29165 ...",normal way,3905.075,44,4785,318,4452,22.615,4200255692,1425375093,167949.278037,2.537375,1.860329,1.451289,2347.591986,3927.690,1547.934469,2111.287850,2706.346193,4785-318
1780755,"MULTILINESTRING ((-6.29133 53.30125, -6.29165 ...",normal way,3730.072,42,4785,318,4453,22.615,4214361944,1425375093,149229.419167,2.391789,1.788471,1.417389,2191.431686,3752.687,1568.987766,2098.265721,2647.605219,4785-318


In [13]:
Entry_geom  = ParkRoads[0].iloc[RoadComb[0].iloc[RoadComb[0].iloc[1076863:1076863+1]['Park_entry_No']]['Park_entry_No']]
entr  = RoadComb[0].iloc[RoadComb[0].iloc[RoadComb[0].iloc[1076863:1076863+1]['Grid_No']]['Grid_No']]
entr    

,index,raw euclidean,size_infl_eucl2,size_infl_eucl3,size_infl_eucl5,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,...,Park_geom,Parkroad_m_centroid,walk_area_m2,size_infl_proot2,size_infl_proot3,size_infl_proot5,grid_osm,geometry,geometry_m,grid_entry_dist
26,4575417,2219.788608,952.321405,1262.678842,1582.342123,1,956,183,POINT (-6.22184 53.40893),POINT (-112243.732 5957469.407),...,"MULTIPOLYGON (((-6.18812 53.39655, -6.18809 53...",POINT (-110161.206 5956700.936),141731.055772,2.330924,1.757999,1.40285,822791806,POINT (-6.22185 53.40795),POINT (-112258.366 5957360.336),110.048


In [14]:
entry = ParkRoads[0].iloc[RoadComb[0].iloc[RoadComb[0].iloc[1076863:1076863+1]['Park_entry_No']]['Park_entry_No']]
nearest = pd.DataFrame((abs(float(entry['geometry_x'].x) - ParkRoads[0]['geometry_x'].x) ** 2
 + abs(float(entry['geometry_x'].y) - ParkRoads[0]['geometry_x'].y) ** 2) ** (1/2)).join(ParkRoads[0]['osmid']).sort_values(0)
nearest[1:10]

,0,osmid
2837,0.000401,6670384106
2825,0.000488,2401021703
2820,0.000887,305242584
2827,0.000943,2401021706
2823,0.001113,305243118
2826,0.001338,2401021704
2819,0.001352,305242514
2824,0.001653,2401021698
2816,0.001856,291661564


In [15]:
# Block 8 determine best parkentry points from each grid, then calculate grid scores
# and finally aggregate city access in categories (high, medium, low and no access)

start_time = time.time()
popg_acc = pd.DataFrame()
adj_popg_acc = pd.DataFrame()
grid_scores = list([])
gridpark = list([])
for n in range(len(cities)):    
    print(cities[n])
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    grid_score = list([])
    gridparks = list([])
    gridpark.append(gridparks)
    popgrid_access = pd.DataFrame()
    adj_popgrid_access = pd.DataFrame()
    for i in range(len(l1)):
        # Get the lowest indices grouped by a key consisting of grid no and park no (best entry point from a grid to a park)
        str1 = 'gridpark_' + l1[i]
        locals()[str1] = Routes[n].iloc[Routes[n].groupby('gridpark_no')[(str(l1[i]) +'_total_cost')].idxmin()]
        l2 = list()
        
        # Get the total cost column
        for j in enumerate(l1): 
            if j[0] != i: l2.append(j[1] + '_total_cost')
        locals()[str1] = locals()[str1].loc[:, ~locals()[str1].columns.isin(l2)]
        
        # Get grid information
        locals()[str1] = pd.merge(locals()[str1], grids[n][['PoP2015_Number','geometry']],
                                left_on = 'Grid_No', right_index = True, how = 'outer')
        locals()[str1] = locals()[str1].reset_index()
        
        # formatting
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].fillna(-1)
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].astype(int)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].fillna(-1)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].astype(int)
        
        grdsc = pd.DataFrame()
        gridsc = pd.DataFrame()
        print(m1[i], round((time.time() - start_time) / 60,2), 'mns')
        
        # For each threshold given, calculate a score
        for k in range(len(thresholds)):
            t = thresholds[k]
            str2 = str(t)
            score = 'tr_'+ str2
            adj_score = 'adj_tr_' + str2
            
            #Only get routes within the threshold given (it loops over every threshold) and calculate the scores
            thold = locals()[str1][locals()[str1][l1[i] + '_total_cost'] <= t]
            thold[score] = t - thold[l1[i] + '_total_cost']
            thold['pop' + score] = thold[score] * thold['PoP2015_Number']
            thold[adj_score] = thold[score] * ((math.pi*600**2) / (math.pi*t**2)) / (600/t)
            thold['walk_area_ha' + str2] = locals()[str1]['walk_area_m2'] /10000
            thold['walkha_person' + str2] = thold['PoP2015_Number'] / thold['walk_area_ha' + str2]
            #thold = thold[thold[score] > 0]
            
            # Join the gridpark information from before.
            locals()[str1] = locals()[str1].join(thold[[score,'pop' + score,adj_score,'walk_area_ha' + str2, 'walkha_person' + str2]])
            # get the grid_scores
            gs = pd.DataFrame()
            gs[[score,'pop_' + score,adj_score,'walkha_' + score,'walkha_person_' + score]] = locals()[str1].groupby(
                    'Grid_No')[score,'pop' + score, adj_score, 'walk_area_ha' + str2, 'walkha_person' + str2].sum()
            
            trstr = locals()[str1][locals()[str1][score] > 0]
            gs[score + '_parks'] = trstr.groupby('Grid_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            gs[score + '_routes'] = gpd.GeoDataFrame(trstr[['Grid_No','geometry_x']],
                                                          geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')

            # Add parks which grids have access to with its closest access point
            gs[score+'Park:entry'] = trstr[trstr['Park_No'] >=0].groupby('Grid_No')['Park_No'].apply(list).astype(str
            ) + ':' + trstr[trstr['Park_entry_No'] >=0].groupby('Grid_No')['Park_entry_No'].apply(list).astype(str)
            
            # determine the thresholds category-score. 
            # High >= threshold (perfect score to one park), medium is above half perfect, 
            # low is below this and no is no access to a park for a certain grid within the threshold given
            gs[score+'_access'] = np.select([gs[score] >= t, (gs[score] < t) & (
            gs[score]>= t/2), (gs[score] < t/2) & (gs[score]> 0), gs[score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])
                        
            gs[score+'_adj_access'] = np.select([gs[adj_score] >= t, (gs[adj_score] < t) & (
            gs[adj_score]>= t/2), (gs[adj_score] < t/2) & (gs[adj_score]> 0), gs[adj_score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])
            
            gs = gs.join(grids[n]['PoP2015_Number'], how = 'outer')
            grdsc = pd.concat([grdsc, gs], axis = 1)
            
            gs = gpd.GeoDataFrame(gs, geometry = score + '_routes', crs = 4326)
            gs.to_file('D:Dumps/Scores output/Grid_lines_shp/gridscore_'+ l1[i] + '_' + str2 + '_' + cities[n] + '.shp')
            
            gsc = gs.loc[:,~gs.columns.isin([score + '_routes'])]
            gridsc = pd.concat([gridsc, gsc])

            # Group according to the categories just created and sum the populations living in those grids
            popgacc = pd.DataFrame()
            popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_access')['PoP2015_Number'].sum()
            popgrid_access = pd.concat([popgrid_access, popgacc],axis=1)   
            
            adj_popgacc = pd.DataFrame()
            adj_popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_adj_access')['PoP2015_Number'].sum()
            adj_popgrid_access = pd.concat([adj_popgrid_access, adj_popgacc],axis=1)   
            print('grid ',t)
        
        grid_score.append(grdsc)
        
        gridsc = gridsc.join(grids[n]['geometry'])
        gridsc = gpd.GeoDataFrame(gridsc, geometry = 'geometry', crs = 4326)
        gridsc.to_file('D:Dumps/Scores output/Grid_geoms_shp/gridscore_'+ l1[i] + '_' + cities[n] + '.shp')
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        gridsc = gridsc.loc[:, gridsc.columns!='geometry']
        gridsc.to_csv('D:/Dumps/Scores output/Grid_csv/gridscore_'+ l1[i] + '_' + cities[n] + '.csv')
        gridparks.append(locals()[str1])
        
    grid_scores.append(grid_score)

    # For each city, divide the population access by group by the total to get its share.
    popgrid_access = popgrid_access / popgrid_access.sum()
    popgrid_access = pd.DataFrame(popgrid_access.unstack())
    popg_acc = pd.concat([popg_acc, popgrid_access], axis = 1)
    
    adj_popgrid_access = adj_popgrid_access / adj_popgrid_access.sum()
    adj_popgrid_access = pd.DataFrame(adj_popgrid_access.unstack())
    adj_popg_acc = pd.concat([adj_popg_acc, adj_popgrid_access], axis = 1)
    
    print(cities[n],'done', round((time.time() - start_time) / 60,2), 'mns')
popg_acc.columns = cities
adj_popg_acc.columns = cities

popg_acc.to_csv('D:/Dumps/Scores output/popgrid_access.csv')
adj_popg_acc.to_csv('D:/Dumps/Scores output/radius-euclidean adjusted popgrid access.csv')

adj_popg_acc    
 

Dublin, Ireland
entrance 0.09 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 3.53 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 7.12 mns
grid  300
grid  600
grid  1000
gravity**(1/5) 7.8 mns
grid  300
grid  600
grid  1000
Dublin, Ireland done 8.22 mns


Dublin, Ireland
entrance_300        1 high           0.297927
                    2 medium         0.115342
                    3 low            0.121176
                    4 no             0.465555
entrance_600        1 high           0.327118
                    2 medium         0.296562
                    3 low            0.225072
                    4 no             0.151248
entrance_1000       1 high           0.487166
                    2 medium         0.300386
                    3 low            0.188698
                    4 no             0.023750
gravity**(1/2)_300  1 high           0.316374
                    2 medium         0.123025
                    3 low            0.119237
                    4 no             0.441364
gravity**(1/2)_600  1 high           0.317755
                    2 medium         0.331788
                    3 low            0.263311
                    4 no             0.087147
gravity**(1/2)_1000 1 high           0.479950
                    2 medium         0.399580
                    3 low            0.117634
                    4 no             0.002835
gravity**(1/3)_300  1 high           0.290307
                    2 medium         0.109536
                    3 low            0.106716
                    4 no             0.493441
gravity**(1/3)_600  1 high           0.241619
                    2 medium         0.342598
                    3 low            0.260960
                    4 no             0.154823
gravity**(1/3)_1000 1 high           0.335131
                    2 medium         0.397725
                    3 low            0.256201
                    4 no             0.010943
gravity**(1/5)_300  1 high           0.278841
                    2 medium         0.114579
                    3 low            0.100416
                    4 no             0.506164
gravity**(1/5)_600  1 high           0.241966
                    2 medium         0.334797
                    3 low            0.242879
                    4 no             0.180358
gravity**(1/5)_1000 1 high           0.341789
                    2 medium         0.368529
                    3 low            0.268735
                    4 no             0.020947

In [16]:
grid_scores[0][0]

,tr_300,pop_tr_300,adj_tr_300,walkha_tr_300,walkha_person_tr_300,tr_300_parks,tr_300_routes,tr_300Park:entry,tr_300_access,tr_300_adj_access,...,pop_tr_1000,adj_tr_1000,walkha_tr_1000,walkha_person_tr_1000,tr_1000_parks,tr_1000_routes,tr_1000Park:entry,tr_1000_access,tr_1000_adj_access,PoP2015_Number
Grid_No,,,,,,,,,,,,,,,,,,,,,
0,0.000,0.000,0.000,0.000000,0.000000,NaN,None,NaN,4 no,4 no,...,0.000,0.0000,0.000000,0.000000,NaN,None,NaN,4 no,4 no,4
1,0.000,0.000,0.000,0.000000,0.000000,NaN,None,NaN,4 no,4 no,...,0.000,0.0000,0.000000,0.000000,NaN,None,NaN,4 no,4 no,4
2,0.000,0.000,0.000,0.000000,0.000000,NaN,None,NaN,4 no,4 no,...,0.000,0.0000,0.000000,0.000000,NaN,None,NaN,4 no,4 no,4
3,0.000,0.000,0.000,0.000000,0.000000,NaN,None,NaN,4 no,4 no,...,28825.398,274.5276,14.890265,4.230952,1.0,"(LINESTRING (-6.2174191 53.410397, -6.2175016 ...",[47]:[977],3 low,3 low,63
4,211.599,16716.321,423.198,16.068526,4.916444,1.0,None,[47]:[952],2 medium,1 high,...,72016.321,546.9594,16.068526,4.916444,1.0,None,[47]:[952],2 medium,2 medium,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4781,236.937,41463.975,473.874,20.945709,8.354933,1.0,"LINESTRING (-6.2890101 53.3038277, -6.2888612 ...",[145]:[2794],2 medium,1 high,...,163963.975,562.1622,20.945709,8.354933,1.0,"LINESTRING (-6.2890101 53.3038277, -6.2888612 ...",[145]:[2794],2 medium,2 medium,175
4782,169.756,8997.068,339.512,20.249579,2.617338,1.0,"(LINESTRING (-6.2941105 53.3029945, -6.2941085...",[145]:[2771],2 medium,1 high,...,46097.068,521.8536,20.249579,2.617338,1.0,"(LINESTRING (-6.2941105 53.3029945, -6.2941085...",[145]:[2771],2 medium,2 medium,53
4783,188.247,14871.513,376.494,22.232292,3.553390,1.0,"(LINESTRING (-6.2910154 53.3027659, -6.2902149...",[145]:[2793],2 medium,1 high,...,70171.513,532.9482,22.232292,3.553390,1.0,"(LINESTRING (-6.2910154 53.3027659, -6.2902149...",[145]:[2793],2 medium,2 medium,79


In [17]:
# Block 9 calculte park scores from previously determined best grid-park routes.

start_time = time.time()
cityparks = list([])
for i in range(len(cities)):
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    parks = list([])
    for j in range(len(l1)):
        parksc = pd.DataFrame()
        prksc = pd.DataFrame()
        for k in range(len(thresholds)):
            score = 'tr_' + str(thresholds[k])
            str2 = str(thresholds[k])
            str1 = gridpark[i][j][gridpark[i][j][score] > 0]

            # Get the park scores
            prk = pd.DataFrame()
            prk[[score,'pop_' + score,'walkha_' + score]] = str1.groupby(
                'Park_No')[score,'pop' + score,'walk_area_ha' + str2].sum()
            prk[score + '_parks'] = str1.groupby('Park_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            prk[score+'route'] = gpd.GeoDataFrame(str1[['Park_No','geometry_x']], 
                             geometry = 'geometry_x', crs = 4326).dissolve('Park_No')
            
            # Add parks which grids have access to with its closest access point
            prk[score+'Grid:Pentry'] = str1[str1['Grid_No'] >=0].groupby('Park_No')['Grid_No'].apply(list).astype(str
            ) + ':' + str1[str1['Park_entry_No'] >=0].groupby('Park_No')['Park_entry_No'].apply(list).astype(str)
            
            # Get all parks, even with no score.
            prk = prk.join(parks_in_range[i].iloc[:,0], how = 'outer')
            prk = prk.loc[:,~prk.columns.isin(['components'])]
            #prk = prk.fillna(-1)
            print('park', thresholds[k])
            
            # Get the park score categories (same as grid score)
            prk[score+'_access'] = np.select([prk[score] >= t, (prk[score] < t) & (
                prk[score]>= t/2), (prk[score] < t/2) & (prk[score]> 0), prk[score] <= 0 | prk[score].isna()],
                ['1 high','2 medium','3 low','4 no'])

            parksc = pd.concat([parksc, prk], axis = 1)
                
            prk = gpd.GeoDataFrame(prk, geometry = score+'route', crs = 4326)
            prk.to_file('D:Dumps/Scores output/Park_lines_shp/parkscore_'+ l1[j] + '_' + str2 + '_' + cities[i] + '.shp')
            
            psc = prk.loc[:,~prk.columns.isin([score + 'route'])]
            prksc = pd.concat([prksc, psc])
            
        parks.append(parksc)
        
        prksc = prksc.join(parks_in_range[i]['park_geom'])
        prksc = gpd.GeoDataFrame(prksc, geometry = 'park_geom', crs = 4326)
        prksc.to_file('D:Dumps/Scores output/Grid_geoms_shp/parkscore_'+ l1[j] + '_' + cities[i] + '.shp')
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        prksc = prksc.loc[:, prksc.columns!='park_geom']
        prksc.to_csv('D:/Dumps/Scores output/Park_csv/parkscore_'+ l1[j] + '_' + cities[i]+ '.csv')

        print(m1[j], round((time.time() - start_time) / 60,2), 'mns')
    cityparks.append(parks)
    print(cities[i],'done', round((time.time() - start_time) / 60,2), 'mns')
pd.DataFrame(cityparks[0][0])

park 300
park 600
park 1000
entrance 0.29 mns
park 300
park 600
park 1000
gravity**(1/2) 3.3 mns
park 300
park 600
park 1000
gravity**(1/3) 3.62 mns
park 300
park 600
park 1000
gravity**(1/5) 3.78 mns
Dublin, Ireland done 3.78 mns


IndexError: list index out of range

In [18]:
# Need to adjust, get the preferred park for each grid
# Block 10 get the preferenced parks for each grid (lowest score) for all distance decay variants.
preference = list([])
for n in enumerate(cities): 
    print(n[1])
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    prefer = list([])
    for j in enumerate(l1):
        pref = list([])
        print(m1[j[0]])
        for k in thresholds:
            score = 'tr_'+ str(k)
            g = gridpark[n[0]][j[0]].iloc[gridpark[n[0]][j[0]].groupby('Grid_No')[score].idxmax().dropna().astype(int)]
            g.join(grids[n[0]]['grid_id'], how = 'outer')
            g.to_csv('D:/Dumps/Scores output/Grid_pref_parks_csv/park-pref_' + j[1] +'-'+ str(k) + '-' + n[1] +'.csv')
            g_lines = gpd.GeoDataFrame(g.loc[:, ~g.columns.isin(['geometry_y'])], geometry = 'geometry_x', crs = 4326)
            g_lines.to_file('D:Dumps/Scores output/Grid_pref_parks_lines/park-pref_' +'-'+ j[1] + str(k) + '-' + n[1] +'.shp')
            g_geoms = gpd.GeoDataFrame(g.loc[:, ~g.columns.isin(['geometry_x'])], geometry = 'geometry_y', crs = 4326)
            g_geoms.to_file('D:Dumps/Scores output/Grid_pref_parks_geoms/park-pref_' +'-'+ j[1] + str(k) + '-' + n[1] +'.shp')
            pref.append(g)
            print('park_prefer',k)
        prefer.append(pref)
        len(pref)
    preference.append(prefer)
print('all done')

Dublin, Ireland
entrance
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/2)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/3)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/5)
park_prefer 300
park_prefer 600
park_prefer 1000
all done


In [ ]:
print(round((time.time() - start) / 60,2),'mns')